In [ ]:
import numpy as np
import pandas as pd
import scipy.signal
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
import cmath
%matplotlib inline

In [ ]:
file_list = ['/home/charles/ECG-Manlio/training data/' + str(i) + '.txt' for i in range(1,31)]

In [ ]:
def read_ECG_from_txt(files):
    n = len(files)
    ecg_info = []
    ecg_signals = np.zeros((n, 12, 5000))
    k = 0 #count file number
    for filename in files:
        with open(filename) as f:
            lines = [line.rstrip('\n') for line in f]
            ecg_info.append(lines[:20])
            ecg_signals[k, :, :].flat[:] = lines[20:]
        k = k + 1
    return ecg_info, ecg_signals

In [ ]:
ecg_info, ecg_signals = read_ECG_from_txt(file_list)

In [ ]:
ecg_signals = np.delete(ecg_signals, [2,3,4,5], 1)

In [ ]:
ecg_signals.shape

In [ ]:
t = np.array(range(5000))*0.002
for j in range(8):
    plt.figure(figsize=[20,10])
    plt.plot(t, ecg_signals[0, j, :])
    plt.show()

In [ ]:
# sg = scipy.signal.savgol_filter(ecg_signals[0,0,:], 333, 1, mode='wrap')
# plt.figure(figsize=[36,6])
# plt.plot(t, sg)
# plt.plot(t, ecg_signals[0,0,:])
# plt.plot(t, ecg_signals[0,0,:] - sg)
# plt.grid()

In [ ]:
# sg = scipy.signal.savgol_filter(ecg_signals[0,6,:], 333, 1)
# plt.figure(figsize=[36,6])
# plt.plot(t, sg)
# plt.plot(t, ecg_signals[0,6,:])
# plt.plot(t, ecg_signals[0,6,:] - sg)
# plt.grid()

In [ ]:
# m = np.median(ecg_signals[0,0,:])
# new_ecg_signals = ecg_signals[0,0,:] - m
# p = np.polyfit(t, new_ecg_signals, 4)
# poly4 = np.polyval(p, t)
# new_ecg_signals = new_ecg_signals - poly4

In [ ]:
# plt.figure(figsize=[36,6])
# plt.plot(t, ecg_signals[0,0,:])
# plt.plot(t, poly4)
# sg4 = scipy.signal.savgol_filter(ecg_signals[0,0,:], 333, 1)
# plt.plot(t, sg4)

In [ ]:
def high_pass_filter(signal, order, cutoff, fs):
    nyq = 0.5 * fs
    normal_cutoff = cutoff/nyq
    b, a = scipy.signal.butter(order, normal_cutoff, btype='high')
    filtered_signal = scipy.signal.filtfilt(b, a, signal)
    return filtered_signal

In [ ]:
# filtered = high_pass_filter(ecg_signals[0,0,:], 5, 1.5, 500)
# plt.figure(figsize=[36,6])
# plt.plot(t, filtered)
# plt.plot(t, ecg_signals[0,0,:])
# plt.grid()
# plt.show()

In [ ]:
# filtered = high_pass_filter(ecg_signals[0,6,:], 5, 1.5, 500)
# plt.figure(figsize=[36,6])
# plt.plot(t, filtered)
# plt.plot(t, ecg_signals[0,6,:])
# plt.grid()
# plt.show()

In [ ]:
# filtered = high_pass_filter(ecg_signals[20,0,:], 5, 1, 500)
# plt.figure(figsize=[36,6])
# plt.plot(t, filtered)
# plt.plot(t, ecg_signals[20,0,:])
# plt.grid()
# plt.show()

In [ ]:
# filtered_ecg_signals = np.zeros((30, 8, 5000))
# n1 = np.zeros((30))

# for i in range(30):
#     n_peaks = []
#     for j in range(8):
#         filtered_signal = high_pass_filter(ecg_signals[i,j,:],5,1,500)
#         m = filtered_signal.max() * 0.5
#         peaks, _ = scipy.signal.find_peaks(filtered_signal, height=m)
#         n_peaks.append(len(peaks))
#     n1[i] = scipy.stats.mode(n_peaks)[0][0]
#     for k in range(8):
#         filtered_ecg_signals[i,k,:] = high_pass_filter(ecg_signals[i,k,:], 5, n1[i]/10, 500)

In [ ]:
# filtered_ecg_signals = np.zeros((30, 8, 5000))
# n2 = np.zeros((30))

# for i in range(30):
#     n_peaks = []
#     for j in range(8):
#         filtered_signal = high_pass_filter(ecg_signals[i,j,:],5,1,500)
#         m = filtered_signal.max() * 0.75
#         peaks, _ = scipy.signal.find_peaks(filtered_signal, height=m)
#         n_peaks.append(len(peaks))
#     n2[i] = scipy.stats.mode(n_peaks)[0][0]
#     for k in range(8):
#         filtered_ecg_signals[i,k,:] = high_pass_filter(ecg_signals[i,k,:], 5, n2[i]/10, 500)

In [ ]:
def autocorr(signal):
    return np.correlate(signal, signal, mode='full')

In [ ]:
def root_sum_square(array):
    return np.sqrt(np.sum(array[:] ** 2)) 

In [ ]:
filtered_ecg_signals = np.zeros((30, 8, 5000))
n = np.zeros((30))

autocorr_t = np.array(range(-4999,5000))*0.002
autocorr_signal = np.zeros((30,8,9999))
autocorr_auc = np.zeros((30,8))

for i in range(30):
    n_peaks = []
    for j in range(8):
        filtered_signal = high_pass_filter(ecg_signals[i,j,:],5,1,500)
        if abs(filtered_signal.min()) > filtered_signal.max():
            filtered_signal = - 1 * filtered_signal
        m = filtered_signal.max() * 0.5
        peaks, _ = scipy.signal.find_peaks(filtered_signal, height=m)
        n_peaks.append(len(peaks))
    n[i] = scipy.stats.mode(n_peaks)[0][0]
    for k in range(8):
        filtered_ecg_signals[i,k,:] = high_pass_filter(ecg_signals[i,k,:], 5, n[i]/10, 500)
        autocorr_signal[i,k,:] = autocorr(filtered_ecg_signals[i,k,:])
        autocorr_signal[i,k,:] = autocorr_signal[i,k,:] / root_sum_square(autocorr_signal[i,k,:])
        autocorr_auc[i,k] = scipy.integrate.simps(abs(autocorr_signal[i,k,:]), autocorr_t)

In [ ]:
n

In [ ]:
for i in range(30):
    plt.plot(autocorr_t, autocorr_signal[i,0,:])
    plt.show()

In [ ]:
def Fourier_transform(freq, time, signal):
    assert isinstance(freq, np.ndarray)
    ft_result = np.zeros(freq.shape, dtype=complex)
    for l in range(len(freq)):
        s = 0
        for k in range(2, len(time)):
            s += ((signal[k] - signal[k-1]) / (time[k] - time[k-1])) * (cmath.exp(-1j*freq[l]*time[k-1]) - cmath.exp(-1j*freq[l]*time[k]))
        ft_signal = - (freq[l] * 1j + (1 - cmath.exp(- 1j * freq[l] * time[1])) * (signal[1] - 1) / time[1] + s) / freq[l]**2
        ft_result[l] = ft_signal
    return np.multiply(ft_result, freq) * 1j

In [ ]:
def cs_oversample(t, signal, factor):
    cs_interpolator = CubicSpline(t, signal)
    num = len(t) * factor
    oversampled_t = np.linspace(0,10,num=num,endpoint=False)
    interpolated_signal = cs_interpolator(oversampled_t)
    return oversampled_t, interpolated_signal

In [ ]:
ft_autocorr_signal = np.zeros((30,8,10000))
freq = np.array([0.1 * w for w in range(1,10001)])

for i in range(30):
    for j in range(8):
        oversampled_t, interpolated_signal = cs_oversample(t, autocorr_signal[i,j,4999:], 4)
        ft_autocorr_signal[i,j] = Fourier_transform(freq, oversampled_t, interpolated_signal)
        print(str(i)+','+str(j)+'finished')

In [ ]:
a,b = cs_oversample(t, autocorr_signal[0,0,4999:],4)

In [ ]:
Fourier_transform(freq, a, b)